## Chapter 15: Calculus and automatic differentiation 

# 15.4  The computation graph

Virtually any function $g$ expressed via a formula can be broken down - akin to the way physical substances may be broken down into their atomic parts - into a combination of elementary functions (e.g., $\text{sin}$, $\text{log}$, etc.,) and operations (e.g., addition, subtraction, etc.,).  One very useful way of organizing the elementary decomposition of a generic function is via a so-called *computation graph*.  The *computation graph* of a function $g$ not only allows us to more easily understand it as a combination of elementury functions and operations, but it also allows us to evaluate a function in a recursive (and therefore very programmatic) way.

In [1]:
# This code cell will not be shown in the HTML version of this notebook
#imports from custom library
import sys
sys.path.append('../../')
import autograd.numpy as np
import matplotlib.pyplot as plt
from mlrefined_libraries import calculus_library as calclib
from mlrefined_libraries import basics_library as baslib

%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

Take the function

\begin{equation}
g(w) = \text{tanh}(w)\text{cos}(w) + \text{log}(w).
\end{equation}

We can decompose this function into its simplest parts, conveniently organized into a *computation graph*, as shown below.  This graphical depiction is like a blueprint, showing us precisely how $g$ is constructed from elementary functions *and* operations.  We read this computation graph *from left to right* starting with the input node representing $w$ and ending with the full computation of $g\left(w\right)$ on the right.  Each *node* (after the input on the left, colored yellow) in the graph represents *a single elementary function or operation*, and is marked as such, and the arrows conecting pairs of nodes are called *directed edges* which show how computation flows when computing a value $g(w)$.  The words *parent* and *child* are often used to describe the local topology of a computation graphs, or in other words two nodes connected in a graph by a single edge.  The parent node is the one taking in the other node as input, or where the arrow points to.  The child node is the input of the parent, or where the arrow originates.  Because these terms / relationships are defined locally a particular node can be both a parent and a child with respect to other nodes in the graph. 



<figure>

<p><img src="../../mlrefined_images/calculus_images/func_2_p0.png" width="70%" height="auto"></p>  
<figcaption> <strong>Figure 1:</strong> An example of acomputation graph for a single-input function.  See text for further details.<em>
</em>
</figcaption>
</figure>

If we were to write out the formula of each *parent node* in terms its children, we would have the list of formulae

\begin{array}
\
a = \text{tanh}(w) \\
b = \text{cos}(w) \\
c = \text{log}(w) \\
d = a\times b \\
e = c + d \\
\end{array} 

where the final formula for $e = g(w)$.  Note how each node is a function of its children.  So we can, for example, write $a$ as $a(w)$, since $w$ is the child node of $a$.  Likewise we can write $d$ as $d(a,b)$, since both $a$ and $b$ are children of $d$. However in the end, if we unwravel the definition of each node, every node is really a function of the input $w$ alone.  In other words, we can think of each node as a funciton of $w$ as well so e.g., the $d$ can be written as a function of its children as $d(a,b)$ or as a function of the input $w$ as $d(w)$.

This way of thinking about a function is called a *computation* graph because it allows us easily compute and visualize how the computation of $g(w)$ is performed.  This computation *flows forward from left to right through the graph recursively in sets of children-parent nodes*, beginning with the substitution of a particular value for $w$ on the left and ending with the final evaluation at this value of $g$ on the right.  As the computation flows the evaluation of each node at the input value of $w$ is computed, and one full forward sweep through the graph computes $g(w)$.

In the figure below we illustrate how the value $g\left(w\right)$ for the value $w = 1.5$ is computed using the computation graph.  Beginning on the left we first substitute in the value $w=1.5$ in the input node.  We then evaluate each of the $w$ node's parent nodes, here computing $a\left(1.5\right) = \text{tanh}(1.5)$, $b\left(1.5\right)=\text{cos}(1.5)$, and $c\left(1.5\right) = \text{log}(1.5)$, illustrated visually in the left panel with the child node highlighted in blue and the parents in red.  Computation next flows to any parent whose children have all been evaluated, here the node $d$, as illustrated in the middle panel which uses the same coloring to denote the child and parent nodes.  Note how in computing $d\left(1.5\right) = a(1.5) \times b(1.5)$ we only need access to its evaluated children, that is $a\left(1.5\right)$ and $b\left(1.5\right)$, which we have indeed already computed.  We then evalute the final parent node in the graph - $e$ - at our desired input value.  Once again to compute $e\left(1.5\right) = c(1.5) + d(1.5)$ we only need access to the evaluations made at its children, here $c\left(1.5\right)$ and $d\left(1.5\right)$, which have already been computed. 

<figure>
<img src="../../mlrefined_images/calculus_images/compgraph_1.png" width="100%" >  
<figcaption> <strong>Figure 2:</strong> <em> Visualizing computation flowing through an example computation graph.  See text for further details.
</em>
</figcaption>
</figure>

Computation graphs can similarly be constructed to decompose multi-input functions as well, and likewise employed to evaluate them.   For example, below we show the computation graph for the simple multi-input quadratic $g\left(\mathbf{w}\right) = w_1^2 + w_2^2$.  The two distinct inputs $w_1$ and $w_2$ here are each represented by a distinct node.  Here computation also flows from left to right, recursively from child to parent nodes, just as with the single input example given above and again one forward sweep through the graph is sufficient to calculate any value $g\left(\mathbf{w}\right)$.

<figure>
<p><img src="../../mlrefined_images/calculus_images/quad_0.png" width="55%" height="auto"></p>
<figcaption> <strong>Figure 3:</strong> The computation graph for a multi-input quadratic function.<em>
</em>
</figcaption>
</figure>

The notion of a computation graph decomposition is quite flexible, as functions can be decomposed in various ways.  Here we have broken two example functions down into their simplest, most elementary parts.  However it is more useful to decompose more sophisticated functions (like e.g., multi-layer perceptrons as detailed in Section 13.1) into computation graphs consisting of more sophisticated elemenatry building blocks like e.g., matrix multiplication, vector-wise functions, etc.,